In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [3]:
import datetime as dt
from dateutil.relativedelta import relativedelta

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy import desc

In [5]:
engine = create_engine("sqlite:///db/Veggie_Fruit_DB.sqlite")

In [6]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [7]:
# We can view all of the classes that automap found
Base.classes.keys()

['fruitexportdest',
 'fruitexportval',
 'fruitimportsource',
 'fruitimportval',
 'veggieexportdest',
 'veggieexportval',
 'veggieimportsource',
 'veggieimportval']

In [8]:
inspector=inspect(engine)
inspector.get_table_names()

['fruitexportdest',
 'fruitexportval',
 'fruitimportsource',
 'fruitimportval',
 'veggieexportdest',
 'veggieexportval',
 'veggieimportsource',
 'veggieimportval']

In [9]:
measurement_columns = inspector.get_columns('veggieexportdest')
for c in measurement_columns:
    print(c['name'])

id
product
flow
country
share
avg
yr15
yr16
yr17
yr18
yr19
lat
lon


In [10]:
# Save references to each table
# FED = FruistExportDestination, FEV =FruitExportValue, FIS =FruitImportSource,FIV=FruitImportValue
# VED =VeggieExportDestination, VEV=VeggieExportValue, VIS=VeggieImportSource, VIV=VeggieImportValue
FED = Base.classes.fruitexportdest
FEV = Base.classes.fruitexportval
FIS = Base.classes.fruitimportsource
FIV = Base.classes.fruitimportval
VED = Base.classes.veggieexportdest
VEV = Base.classes.veggieexportval
VIS = Base.classes.veggieimportsource
VIV = Base.classes.veggieimportval


In [11]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [12]:
# route 1 Generate Summary Export(for summary_edata route)
fe_summary = session.query(FED.country,FED.yr17,FED.yr18).group_by(FED.country).order_by(FED.yr18.desc()).all()
fe_summary_df = pd.DataFrame(fe_summary, columns=['country', 'yr17', 'yr18'])
fe_summary_df['yr17'] = fe_summary_df['yr17'].str.replace(',', '')
fe_summary_df['yr18'] = fe_summary_df['yr18'].str.replace(',', '')
fe_summary_df=fe_summary_df.dropna(how="any")
fe_summary_df["yr17"] = pd.to_numeric(fe_summary_df["yr17"])
fe_summary_df["yr18"] = pd.to_numeric(fe_summary_df["yr18"])

ve_summary = session.query(VED.country,VED.yr17,VED.yr18).group_by(VED.country).order_by(VED.yr18.desc()).all()
ve_summary_df = pd.DataFrame(ve_summary, columns=['country', 'yr17', 'yr18'])
ve_summary_df['yr17'] = ve_summary_df['yr17'].str.replace(',', '')
ve_summary_df['yr18'] = ve_summary_df['yr18'].str.replace(',', '')
ve_summary_df=ve_summary_df.dropna(how="any")
ve_summary_df["yr17"] = pd.to_numeric(ve_summary_df["yr17"])
ve_summary_df["yr18"] = pd.to_numeric(ve_summary_df["yr18"])
esummary_frames = [ve_summary_df, fe_summary_df]
summary_export = pd.concat(esummary_frames,ignore_index=False)
summary_export = summary_export.sort_values(['yr17', 'yr18'], ascending=False)

export_yr17sum=summary_export['yr17'].sum()
export_yr18sum=summary_export['yr18'].sum()

for i in summary_export:
    e_ms17=summary_export['yr17']/export_yr17sum*100

for i in summary_export:
    e_ms18=summary_export['yr18']/export_yr18sum*100

summary_export["MarketShare(2017)"]=e_ms17
summary_export["MarketShare(2018)"]=e_ms18
e_change = e_ms18-e_ms17
summary_export["ChangeInMarketShare"]=e_change
renamed_summary_export = summary_export.rename(columns={"country":"Country", "MS 2017":"MarketShare(2017)","MS 2018":"MarketShare(2018)", "Change in MS":"ChangeInMarketShare","yr17":"Value2017", "yr18":"Value2018"})
summary_export = renamed_summary_export[["Country","MarketShare(2017)","MarketShare(2018)","ChangeInMarketShare", "Value2017", "Value2018"]]
summary_export=summary_export.sort_values(['MarketShare(2018)'], ascending=False).head()
Summary_Export = summary_export[["Country", "MarketShare(2017)", "MarketShare(2018)", "ChangeInMarketShare", "Value2017", "Value2018"]]
Summary_Export["MarketShare(2017)"]= Summary_Export["MarketShare(2017)"].astype(int)
Summary_Export["MarketShare(2018)"]= Summary_Export["MarketShare(2018)"].astype(int)
Summary_Export["ChangeInMarketShare"]= Summary_Export["ChangeInMarketShare"].astype(int)

summary_export_list = []

Summary_Export = Summary_Export.reset_index(drop=True)
ms17_list = Summary_Export["MarketShare(2017)"].tolist()
ms18_list = Summary_Export["MarketShare(2018)"].tolist()
mschange_list = Summary_Export["ChangeInMarketShare"].tolist()
value_yr17 = Summary_Export["Value2017"].tolist()
value_yr18 = Summary_Export["Value2018"].tolist()

country_list = Summary_Export["Country"].tolist()
for i in range(len(country_list)):
    summary_report_dict = {}
    summary_report_dict["Country"] = country_list[i]
    summary_report_dict["MS17"] = ms17_list[i]
    summary_report_dict["MS18"] = ms18_list[i]
    summary_report_dict["MSChange"] = mschange_list[i]
    summary_report_dict["Value2017"] = value_yr17[i]
    summary_report_dict["Value2018"] = value_yr18[i]

    summary_export_list.append(summary_report_dict)

print(summary_export_list)
# return jsonify(summary_export_list)


[{'Country': 'Mexico', 'MS17': 25, 'MS18': 22, 'MSChange': -3, 'Value2017': 289339, 'Value2018': 175074}, {'Country': 'Canada', 'MS17': 14, 'MS18': 13, 'MSChange': -1, 'Value2017': 167337, 'Value2018': 107007}, {'Country': 'India', 'MS17': 15, 'MS18': 11, 'MSChange': -3, 'Value2017': 174179, 'Value2018': 90591}, {'Country': 'Taiwan', 'MS17': 5, 'MS18': 8, 'MSChange': 2, 'Value2017': 67184, 'Value2018': 64703}, {'Country': 'Japan', 'MS17': 2, 'MS18': 4, 'MSChange': 2, 'Value2017': 33401, 'Value2018': 39167}]


In [13]:
# route 2 Generate Top10 Export for @app.route("/top10_edata")

ExportDataVeggie = session.query(VED.country,VED.product,VED.yr17,VED.yr18).\
group_by(VED.country).order_by(VED.yr18.desc()).all()
# # Save the query results as a Pandas DataFrame and set the index to the date column
ve_data_df = pd.DataFrame(ExportDataVeggie, columns=['country','product', 'yr17', 'yr18'])
# Sort the dataframe by date
# all_import_data_df = all_import_data_df.sort_values("country")
# precip_data_df.set_index(precip_data_df['date'], inplace=False)
ve_data_df['yr17'] = ve_data_df['yr17'].str.replace(',', '')
ve_data_df['yr18'] = ve_data_df['yr18'].str.replace(',', '')
ve_data_df = ve_data_df.dropna(how="any")
ve_data_df["yr17"] = pd.to_numeric(ve_data_df["yr17"])
ve_data_df["yr18"] = pd.to_numeric(ve_data_df["yr18"])
ve_data_df.sort_values(['yr17', 'yr18'], ascending=False).head()

ExportDataFruit = session.query(FED.country,FED.product,FED.yr17,FED.yr18).\
group_by(FED.country).order_by(FED.yr17.desc()).all()
fe_data_df = pd.DataFrame(ExportDataFruit, columns=['country','product', 'yr17', 'yr18'])
fe_data_df['yr17'] = fe_data_df['yr17'].str.replace(',', '')
fe_data_df['yr18'] = fe_data_df['yr18'].str.replace(',', '')
fe_data_df["yr17"] = pd.to_numeric(fe_data_df["yr17"])
fe_data_df["yr18"] = pd.to_numeric(fe_data_df["yr18"])
fe_data_df = fe_data_df.dropna(how="any")
fe_data_df.sort_values(['yr17', 'yr18'], ascending=False)
export_frames = [ve_data_df, fe_data_df]
all_export = pd.concat(export_frames)
all_export.sort_values(['yr17', 'yr18'], ascending=False).head()
all_export_yr18sum=all_export['yr18'].sum()
for i in all_export:
    eproduct_share=all_export["yr18"]/all_export_yr18sum*100
all_export["Share"]=eproduct_share
organized_all_export = all_export[["country","product","yr18","Share"]]
org_top10_export = organized_all_export.sort_values(['Share'], ascending=False).head(10)
top10_export = org_top10_export.rename(columns={"country":"Country", "product":"Product","yr18":"Value2018", "Share":"Share"})
top10_export.reset_index(inplace=True)
Top10_Export= top10_export[["Country", "Product", "Value2018", "Share"]] 

Top10_Export["Share"]= Top10_Export["Share"].astype(int)

top10_export_list =[]
top10country_list = Top10_Export["Country"].tolist()
top10product_list = Top10_Export["Product"].tolist()
top10value_yr18 = Top10_Export["Value2018"].tolist()
top10ms18_list = Top10_Export["Share"].tolist()

for i in range(len(top10country_list)):
    top10_export_dict = {}
    top10_export_dict["Country"] = top10country_list[i]
    top10_export_dict["Product"] = top10product_list[i]
    top10_export_dict["Value2018"] = top10value_yr18[i]
    top10_export_dict["Share"] = top10ms18_list[i]

    top10_export_list.append(top10_export_dict)

print(top10_export_list)
# return jsonify(top10_export_list)


[{'Country': 'Mexico', 'Product': 'Apple', 'Value2018': 175074.0, 'Share': 22}, {'Country': 'Canada', 'Product': 'Apple', 'Value2018': 107007.0, 'Share': 13}, {'Country': 'India', 'Product': 'Apple', 'Value2018': 90591.0, 'Share': 11}, {'Country': 'Taiwan', 'Product': 'Apple', 'Value2018': 64703.0, 'Share': 8}, {'Country': 'Japan', 'Product': 'Broccoli', 'Value2018': 39167.0, 'Share': 4}, {'Country': 'Vietnam', 'Product': 'Apple', 'Value2018': 37473.0, 'Share': 4}, {'Country': 'Hong Kong', 'Product': 'Apple', 'Value2018': 27545.0, 'Share': 3}, {'Country': 'Indonesia', 'Product': 'Apple', 'Value2018': 22832.0, 'Share': 2}, {'Country': 'United Arab Emirates', 'Product': 'Apple', 'Value2018': 21264.0, 'Share': 2}, {'Country': 'Saudi Arabia', 'Product': 'Apple', 'Value2018': 17625.0, 'Share': 2}]


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
# route 3 Generate Export Bar_Pie Chart data for # @app.route("/bar_edata")

ExportDataVeggie = session.query(VED.product,VED.yr18).\
group_by(VED.product).order_by(VED.yr18.desc()).all()
# # Save the query results as a Pandas DataFrame and set the index to the date column
ve_data_df = pd.DataFrame(ExportDataVeggie, columns=['product', 'yr18'])
# Sort the dataframe by date
# all_import_data_df = all_import_data_df.sort_values("country")
# precip_data_df.set_index(precip_data_df['date'], inplace=False)

ve_data_df['yr18'] = ve_data_df['yr18'].str.replace(',', '')
ve_data_df = ve_data_df.dropna(how="any")
ve_data_df["yr18"] = pd.to_numeric(ve_data_df["yr18"])
ve_data_df.sort_values(['yr18'], ascending=False).head()

ExportDataFruit = session.query(FED.product,FED.yr18).\
group_by(FED.product).order_by(FED.yr18.desc()).all()
fe_data_df = pd.DataFrame(ExportDataFruit, columns=['product','yr18'])
fe_data_df['yr18'] = fe_data_df['yr18'].str.replace(',', '')
fe_data_df["yr18"] = pd.to_numeric(fe_data_df["yr18"])
fe_data_df = fe_data_df.dropna(how="any")
fe_data_df.sort_values(['yr18'], ascending=False)
export_frames = [ve_data_df, fe_data_df]
all_export = pd.concat(export_frames)
all_export.sort_values(['yr18'], ascending=False).head()

organized_bar_export = all_export[["product","yr18"]]
b_top10_export = organized_bar_export.sort_values(['yr18'], ascending=False).head(10)
b_top10_export = b_top10_export.rename(columns={"product":"Product","yr18":"Value2018"})
b_top10_export.reset_index(inplace=True)
bTop10_Export= b_top10_export[["Product", "Value2018"]] 

# Top10_Export["Share"]= Top10_Export["Share"].astype(int)

top10_export_blist =[]
# top10country_list = Top10_Export["Country"].tolist()
top10product_blist = bTop10_Export["Product"].tolist()
top10value_byr18 = bTop10_Export["Value2018"].tolist()

for i in range(len(top10product_blist)):
    top10_export_bdict = {}
    top10_export_bdict["Product"] = top10product_blist[i]
    top10_export_bdict["Value2018"] = top10value_byr18[i]


    top10_export_blist.append(top10_export_bdict)


data = {
       "bproduct": top10product_blist,
       "bvalue": top10value_byr18,
   }
print(data)
# return jsonify(data)


{'bproduct': ['Grapes', 'Apple', 'Oranges', 'Carrot', 'Potato', 'Sweet Corn', 'Broccoli', 'Onion', 'Strawberries', 'Pears'], 'bvalue': [221177.0, 175074.0, 167600.0, 154315.0, 141898.0, 134033.0, 113061.0, 80601.0, 60943.0, 57001.0]}


In [15]:
# 4 Generate Summary Import(for summary_idata route)
fi_summary = session.query(FIS.country,FIS.yr17,FIS.yr18).group_by(FIS.country).order_by(FIS.yr18.desc()).all()
fi_summary_df = pd.DataFrame(fi_summary, columns=['country', 'yr17', 'yr18'])
fi_summary_df['yr17'] = fi_summary_df['yr17'].str.replace(',', '')
fi_summary_df['yr18'] = fi_summary_df['yr18'].str.replace(',', '')
fi_summary_df=fi_summary_df.dropna(how="any")
fi_summary_df["yr17"] = pd.to_numeric(fi_summary_df["yr17"])
fi_summary_df["yr18"] = pd.to_numeric(fi_summary_df["yr18"])

vi_summary = session.query(VIS.country,VIS.yr17,VIS.yr18).group_by(VIS.country).order_by(VIS.yr18.desc()).all()
vi_summary_df = pd.DataFrame(vi_summary, columns=['country', 'yr17', 'yr18'])
vi_summary_df['yr17'] = vi_summary_df['yr17'].str.replace(',', '')
vi_summary_df['yr18'] = vi_summary_df['yr18'].str.replace(',', '')
vi_summary_df=vi_summary_df.dropna(how="any")
vi_summary_df["yr17"] = pd.to_numeric(vi_summary_df["yr17"])
vi_summary_df["yr18"] = pd.to_numeric(vi_summary_df["yr18"])
isummary_frames = [vi_summary_df, fi_summary_df]
summary_import = pd.concat(isummary_frames,ignore_index=False)
summary_import = summary_import.sort_values(['yr17', 'yr18'], ascending=False)

import_yr17sum=summary_import['yr17'].sum()
import_yr18sum=summary_import['yr18'].sum()

for i in summary_import:
    i_ms17=summary_import['yr17']/import_yr17sum*100

for i in summary_import:
    i_ms18=summary_import['yr18']/import_yr18sum*100

summary_import["MarketShare(2017)"]=i_ms17
summary_import["MarketShare(2018)"]=i_ms18
i_change = i_ms18-i_ms17
summary_import["ChangeInMarketShare"]=i_change
renamed_summary_import = summary_import.rename(columns={"country":"Country", "MS 2017":"MarketShare(2017)","MS 2018":"MarketShare(2018)", "Change in MS":"ChangeInMarketShare","yr17":"Value2017", "yr18":"Value2018"})
summary_import = renamed_summary_import[["Country","MarketShare(2017)","MarketShare(2018)","ChangeInMarketShare", "Value2017", "Value2018"]]
summary_import=summary_import.sort_values(['MarketShare(2018)'], ascending=False).head()
Summary_Import = summary_import[["Country", "MarketShare(2017)", "MarketShare(2018)", "ChangeInMarketShare", "Value2017", "Value2018"]]
Summary_Import["MarketShare(2017)"]= Summary_Import["MarketShare(2017)"].astype(int)
Summary_Import["MarketShare(2018)"]= Summary_Import["MarketShare(2018)"].astype(int)
Summary_Import["ChangeInMarketShare"]= Summary_Import["ChangeInMarketShare"].astype(int)

summary_import_list = []

Summary_Import = Summary_Import.reset_index(drop=True)
ms17_ilist = Summary_Import["MarketShare(2017)"].tolist()
ms18_ilist = Summary_Import["MarketShare(2018)"].tolist()
mschange_ilist = Summary_Import["ChangeInMarketShare"].tolist()
ivalue_yr17 = Summary_Import["Value2017"].tolist()
ivalue_yr18 = Summary_Import["Value2018"].tolist()

country_ilist = Summary_Import["Country"].tolist()
for i in range(len(country_ilist)):
    summary_report_idict = {}
    summary_report_idict["Country"] = country_ilist[i]
    summary_report_idict["MS17"] = ms17_ilist[i]
    summary_report_idict["MS18"] = ms18_ilist[i]
    summary_report_idict["MSChange"] = mschange_ilist[i]
    summary_report_idict["Value2017"] = ivalue_yr17[i]
    summary_report_idict["Value2018"] = ivalue_yr18[i]

    summary_import_list.append(summary_report_idict)

print(summary_import_list)
# return jsonify(summary_export_list)


[{'Country': 'Chile', 'MS17': 31, 'MS18': 30, 'MSChange': 0, 'Value2017': 104232, 'Value2018': 90268}, {'Country': 'New Zealand', 'MS17': 23, 'MS18': 23, 'MSChange': 0, 'Value2017': 77678, 'Value2018': 70271}, {'Country': 'Israel', 'MS17': 12, 'MS18': 14, 'MSChange': 1, 'Value2017': 42295, 'Value2018': 42982}, {'Country': 'Dominican Republic', 'MS17': 9, 'MS18': 12, 'MSChange': 2, 'Value2017': 32344, 'Value2018': 36660}, {'Country': 'Canada', 'MS17': 11, 'MS18': 8, 'MSChange': -2, 'Value2017': 37604, 'Value2018': 24737}]


In [16]:
# route 5 Generate Top10 Import for @app.route("/top10_idata")

ImportDataVeggie = session.query(VIS.country,VIS.product,VIS.yr17,VIS.yr18).\
group_by(VIS.country).order_by(VIS.yr18.desc()).all()
# # Save the query results as a Pandas DataFrame and set the index to the date column
vi_data_df = pd.DataFrame(ImportDataVeggie, columns=['country','product', 'yr17', 'yr18'])
# Sort the dataframe by date
# all_import_data_df = all_import_data_df.sort_values("country")
# precip_data_df.set_index(precip_data_df['date'], inplace=False)
vi_data_df['yr17'] = vi_data_df['yr17'].str.replace(',', '')
vi_data_df['yr18'] = vi_data_df['yr18'].str.replace(',', '')
vi_data_df = vi_data_df.dropna(how="any")
vi_data_df["yr17"] = pd.to_numeric(vi_data_df["yr17"])
vi_data_df["yr18"] = pd.to_numeric(vi_data_df["yr18"])
vi_data_df.sort_values(['yr17', 'yr18'], ascending=False).head()

ImportDataFruit = session.query(FIS.country,FIS.product,FIS.yr17,FIS.yr18).\
group_by(FIS.country).order_by(FIS.yr17.desc()).all()
fi_data_df = pd.DataFrame(ImportDataFruit, columns=['country','product', 'yr17', 'yr18'])
fi_data_df['yr17'] = fi_data_df['yr17'].str.replace(',', '')
fi_data_df['yr18'] = fi_data_df['yr18'].str.replace(',', '')
fi_data_df["yr17"] = pd.to_numeric(fi_data_df["yr17"])
fi_data_df["yr18"] = pd.to_numeric(fi_data_df["yr18"])
fi_data_df = fi_data_df.dropna(how="any")
fi_data_df.sort_values(['yr17', 'yr18'], ascending=False)
import_frames = [vi_data_df, fi_data_df]
all_import = pd.concat(import_frames)
all_import.sort_values(['yr17', 'yr18'], ascending=False).head()
all_import_yr18sum=all_import['yr18'].sum()
for i in all_import:
    iproduct_share=all_import["yr18"]/all_import_yr18sum*100
all_import["Share"]=iproduct_share
organized_all_import = all_import[["country","product","yr18","Share"]]
org_top10_import = organized_all_import.sort_values(['Share'], ascending=False).head(10)
top10_import = org_top10_import.rename(columns={"country":"Country", "product":"Product","yr18":"Value2018", "Share":"Share"})
top10_import.reset_index(inplace=True)
Top10_Import= top10_import[["Country", "Product", "Value2018", "Share"]] 

Top10_Import["Share"]= Top10_Import["Share"].astype(int)

top10_import_list =[]
top10country_ilist = Top10_Import["Country"].tolist()
top10product_ilist = Top10_Import["Product"].tolist()
top10value_iyr18 = Top10_Import["Value2018"].tolist()
top10ms18_ilist = Top10_Import["Share"].tolist()

for i in range(len(top10country_ilist)):
    top10_import_dict = {}
    top10_import_dict["Country"] = top10country_ilist[i]
    top10_import_dict["Product"] = top10product_ilist[i]
    top10_import_dict["Value2018"] = top10value_iyr18[i]
    top10_import_dict["Share"] = top10ms18_ilist[i]

    top10_import_list.append(top10_import_dict)

print(top10_import_list)
# return jsonify(top10_export_list)


[{'Country': 'Chile', 'Product': 'Apple', 'Value2018': 90268.0, 'Share': 30}, {'Country': 'New Zealand', 'Product': 'Apple', 'Value2018': 70271.0, 'Share': 23}, {'Country': 'Israel', 'Product': 'Carrot', 'Value2018': 42982.0, 'Share': 14}, {'Country': 'Dominican Republic', 'Product': 'Avocado', 'Value2018': 36660.0, 'Share': 12}, {'Country': 'Canada', 'Product': 'Apple', 'Value2018': 24737.0, 'Share': 8}, {'Country': 'Argentina', 'Product': 'Apple', 'Value2018': 10913.0, 'Share': 3}, {'Country': 'China', 'Product': 'Apple', 'Value2018': 5665.0, 'Share': 1}, {'Country': 'China', 'Product': 'Mushroom', 'Value2018': 4753.0, 'Share': 1}, {'Country': 'Uruguay', 'Product': 'Blueberries', 'Value2018': 4159.0, 'Share': 1}, {'Country': 'Greece', 'Product': 'Figs', 'Value2018': 3042.0, 'Share': 1}]


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [17]:
# 6 Generate Import Bar_Pie Chart data for # @app.route("/bar_idata")

bImportDataVeggie = session.query(VIS.country,VIS.product,VIS.yr18).\
group_by(VIS.country).order_by(VIS.yr18.desc()).all()
# # Save the query results as a Pandas DataFrame and set the index to the date column
bvi_data_df = pd.DataFrame(bImportDataVeggie, columns=['country','product','yr18'])
# Sort the dataframe by date
# all_import_data_df = all_import_data_df.sort_values("country")
# precip_data_df.set_index(precip_data_df['date'], inplace=False)

bvi_data_df['yr18'] = bvi_data_df['yr18'].str.replace(',', '')
bvi_data_df = bvi_data_df.dropna(how="any")
bvi_data_df["yr18"] = pd.to_numeric(bvi_data_df["yr18"])
bvi_data_df.sort_values(['yr18'], ascending=False).head()

bImportDataFruit = session.query(FIS.country,FIS.product,FIS.yr18).\
order_by(FIS.yr18.desc()).all()
bfi_data_df = pd.DataFrame(bImportDataFruit, columns=['country','product', 'yr18'])
bfi_data_df['yr18'] = bfi_data_df['yr18'].str.replace(',', '')
bfi_data_df["yr18"] = pd.to_numeric(bfi_data_df["yr18"])
bfi_data_df = bfi_data_df.dropna(how="any")
bfi_data_df.sort_values(['yr18'], ascending=False)
bimport_frames = [bvi_data_df, bfi_data_df]
ball_import = pd.concat(bimport_frames)
ball_import.sort_values(['yr18'], ascending=False).head()
borganized_all_import = ball_import[["country","product","yr18"]]
borg_top10_import = borganized_all_import.sort_values(['yr18'], ascending=False).head(10)
btop10_import = borg_top10_import.rename(columns={"country":"Country", "product":"Product","yr18":"Value2018"})
# btop10_export.reset_index(inplace=True)
bTop10_Import= btop10_import[["Country", "Product", "Value2018"]] 
bTop10_Import = bTop10_Import.reset_index(drop=True)

btop10_import_list =[]
btop10country_ilist = bTop10_Import["Country"].tolist()
btop10product_ilist = bTop10_Import["Product"].tolist()
btop10value_iyr18 = bTop10_Import["Value2018"].tolist()

for i in range(len(btop10country_ilist)):
    btop10_import_dict = {}
    btop10_import_dict["Country"] = btop10country_ilist[i]
    btop10_import_dict["Product"] = btop10product_ilist[i]
    btop10_import_dict["Value2018"] = btop10value_iyr18[i]


    btop10_import_list.append(btop10_import_dict)


data1 = {
       "bcountry": btop10country_ilist, 
       "bproduct": btop10product_ilist,
       "bvalue": btop10value_iyr18,
   }

print(data1)
# return jsonify(data)

{'bcountry': ['Mexico', 'Chile', 'Mexico', 'Costa Rica', 'Mexico', 'Mexico', 'Peru', 'Chile', 'Mexico', 'Peru'], 'bproduct': ['Avocado', 'Grapes', 'Strawberries', 'Pineapples', 'Grapes', 'Limes', 'Grapes', 'Blueberries', 'Blueberries', 'Blueberries'], 'bvalue': [2071943.0, 778594.0, 584893.0, 537414.0, 511061.0, 416976.0, 360141.0, 354916.0, 289616.0, 284433.0]}


In [18]:
# @app.route("/veggienames")
    
market_year = session.query(VED.product , VED.country , VED.share).all()
mkt_yr_df = pd.DataFrame(market_year, columns = ["Product","Country","Share"])
product_group = mkt_yr_df.groupby(["Product"]).count()
product_list = product_group.index.values.tolist()
product_list = product_list[1:]

print(product_list)
# return jsonify(product_list)



['Beet', 'Broccoli', 'Carrot', 'Green Pea', 'Mushroom', 'Onion', 'Pepper', 'Potato', 'Sweet Corn']


In [22]:
# @app.route("/veggieexp/<veggie>")

sel = [
    VED.product,
    VED.country,
    VED.share,
    VED.lat,
    VED.lon,
    ]
veggie = VED.product
results = session.query(*sel).filter(VED.product == veggie).all()

veggie_list = []
for result in results:
    veggie_country = {}
    veggie_country["Country"] = result[1]
    veggie_country["Type"] = "veggie"
    veggie_country["Share"] = float(result[2].replace('%',''))
    veggie_country["lat"] = result[3]
    veggie_country["lon"] = result[4]
    veggie_list.append(veggie_country)

print(veggie_list)
# return jsonify(veggie_list)

[{'Country': 'Canada', 'Type': 'veggie', 'Share': 69.9, 'lat': 56.130366, 'lon': -106.346771}, {'Country': 'Mexico', 'Type': 'veggie', 'Share': 21.3, 'lat': 23.634501, 'lon': -102.552784}, {'Country': 'Peru', 'Type': 'veggie', 'Share': 1.6, 'lat': -9.189967, 'lon': -75.015152}, {'Country': 'Aruba', 'Type': 'veggie', 'Share': 1.4, 'lat': 12.52111, 'lon': -69.968338}, {'Country': 'Panama', 'Type': 'veggie', 'Share': 1.4, 'lat': 8.537981, 'lon': -80.782127}, {'Country': 'Canada', 'Type': 'veggie', 'Share': 97.7, 'lat': 56.130366, 'lon': -106.346771}, {'Country': 'Thailand', 'Type': 'veggie', 'Share': 0.8, 'lat': 15.870032, 'lon': 100.992541}, {'Country': 'Mexico', 'Type': 'veggie', 'Share': 0.5, 'lat': 23.634501, 'lon': -102.552784}, {'Country': 'Federal Republic of Germany', 'Type': 'veggie', 'Share': 0.3, 'lat': 51.165691, 'lon': 10.451526}, {'Country': 'Trinidad and Tobago', 'Type': 'veggie', 'Share': 0.2, 'lat': 10.691803, 'lon': -61.222503}, {'Country': 'Canada', 'Type': 'veggie', 'S

In [23]:
# @app.route("/fruitnames")
    
fruit_specific = session.query(FED.product , FED.country , FED.share).all()
fruit_spec_df = pd.DataFrame(fruit_specific, columns = ["Product","Country","Share"])
fruit_group = fruit_spec_df.groupby(["Product"]).count()
fruit_overall = fruit_group.index.values.tolist()

print(fruit_overall)
# return jsonify(fruit_overall)



['Apple', 'Apricot', 'Avocado', 'Blueberries', 'Cherries', 'Citrus', 'Cranberries', 'Figs', 'Grapefruit', 'Grapes', 'Guavas', 'Kiwi', 'Limes', 'Oranges', 'Papaya', 'Peaches', 'Pears', 'Persimmons', 'Pineapples', 'Plums', 'Strawberries', 'Tangerines']


In [24]:
# @app.route("/fruitexp/<fruit>")

sel = [
    FED.product,
    FED.country,
    FED.share,
    FED.lat,
    FED.lon,
    ]
fruit = FED.product
results = session.query(*sel).filter(FED.product == fruit).all()
fruit_list = []
for result in results:
    fruit_country = {}
    fruit_country["Country"] = result[1]
    fruit_country["Type"] = "Fruit"
    fruit_country["Share"] = float(result[2].replace('%',''))
    fruit_country["lat"] = result[3]
    fruit_country["lon"] = result[4]
    fruit_list.append(fruit_country)

print(fruit_list)



[{'Country': 'Mexico', 'Type': 'Fruit', 'Share': 26.4, 'lat': 23.634501, 'lon': -102.552784}, {'Country': 'Canada', 'Type': 'Fruit', 'Share': 17.1, 'lat': 56.130366, 'lon': -106.346771}, {'Country': 'India', 'Type': 'Fruit', 'Share': 10.1, 'lat': 20.593684, 'lon': 78.96288}, {'Country': 'Taiwan', 'Type': 'Fruit', 'Share': 8.1, 'lat': 23.69781, 'lon': 120.960515}, {'Country': 'Vietnam', 'Type': 'Fruit', 'Share': 4.4, 'lat': 14.058324, 'lon': 108.277199}, {'Country': 'Hong Kong', 'Type': 'Fruit', 'Share': 4.0, 'lat': 22.3193039, 'lon': 114.1693611}, {'Country': 'Indonesia', 'Type': 'Fruit', 'Share': 3.6, 'lat': -0.789275, 'lon': 113.921327}, {'Country': 'United Arab Emirates', 'Type': 'Fruit', 'Share': 3.4, 'lat': 23.424076, 'lon': 53.847818}, {'Country': 'Saudi Arabia', 'Type': 'Fruit', 'Share': 2.3, 'lat': 23.885942, 'lon': 45.079162}, {'Country': 'Dominican Republic', 'Type': 'Fruit', 'Share': 2.1, 'lat': 18.735693, 'lon': -70.162651}, {'Country': 'Thailand', 'Type': 'Fruit', 'Share':

In [26]:

# @app.route("/fruitexp/<fruit>")
# def fruit_country(fruit):

sel = [
    FED.product,
    FED.country,
    FED.share,
    FED.lat,
    FED.lon,
]

fruit = FED.product
results = session.query(*sel).filter(FED.product == fruit).all()
fruit_list = []
for result in results:
    fruit_country = {}
    fruit_country["Country"] = result[1]
    fruit_country["Type"] = "Fruit"
    fruit_country["Share"] = float(result[2].replace('%',''))
    fruit_country["lat"] = result[3]
    fruit_country["lon"] = result[4]
    fruit_list.append(fruit_country)

print(fruit_list)


[{'Country': 'Mexico', 'Type': 'Fruit', 'Share': 26.4, 'lat': 23.634501, 'lon': -102.552784}, {'Country': 'Canada', 'Type': 'Fruit', 'Share': 17.1, 'lat': 56.130366, 'lon': -106.346771}, {'Country': 'India', 'Type': 'Fruit', 'Share': 10.1, 'lat': 20.593684, 'lon': 78.96288}, {'Country': 'Taiwan', 'Type': 'Fruit', 'Share': 8.1, 'lat': 23.69781, 'lon': 120.960515}, {'Country': 'Vietnam', 'Type': 'Fruit', 'Share': 4.4, 'lat': 14.058324, 'lon': 108.277199}, {'Country': 'Hong Kong', 'Type': 'Fruit', 'Share': 4.0, 'lat': 22.3193039, 'lon': 114.1693611}, {'Country': 'Indonesia', 'Type': 'Fruit', 'Share': 3.6, 'lat': -0.789275, 'lon': 113.921327}, {'Country': 'United Arab Emirates', 'Type': 'Fruit', 'Share': 3.4, 'lat': 23.424076, 'lon': 53.847818}, {'Country': 'Saudi Arabia', 'Type': 'Fruit', 'Share': 2.3, 'lat': 23.885942, 'lon': 45.079162}, {'Country': 'Dominican Republic', 'Type': 'Fruit', 'Share': 2.1, 'lat': 18.735693, 'lon': -70.162651}, {'Country': 'Thailand', 'Type': 'Fruit', 'Share':